In [1]:
import random

In [1]:
import os, glob, shutil

In [2]:
train_list = []

In [ ]:
lst = [
    "train2007",
    "train2012",
    "val2007",
    "val2012",
]

train_list = [
    f"./images/{folder}/{x}"
    for folder in lst
    for x in os.listdir(os.path.join("/home/setupishe/datasets/VOC/images", folder))
]

In [8]:
test_list = [
    f"./images/test2007/{x}"
    for x in os.listdir("/home/setupishe/datasets/VOC/images/test2007")
]

In [10]:
with open("/home/setupishe/datasets/VOC/VOC_train.txt", "w") as f:
    f.writelines([x + "\n" for x in train_list])

In [11]:
with open("/home/setupishe/datasets/VOC/VOC_val.txt", "w") as f:
    f.writelines([x + "\n" for x in test_list])

In [2]:
import os, glob, shutil

In [3]:
from PIL import Image

In [4]:
import cv2

In [5]:
def get_shape(img_path):
    with Image.open(img_path) as img:
        width, height = img.size
    return height, width

In [6]:
from tqdm import tqdm

In [7]:
def force_mkdir(directory):
    if os.path.isdir(directory):
        shutil.rmtree(directory)
    os.mkdir(directory)

In [11]:
import os, shutil, glob

In [13]:
def jpg2txt(inp):
    return inp.replace("/images", "/labels").replace(".jpg", ".txt")


def txt2jpg(inp):
    return inp.replace("/labels", "/images").replace(".txt", ".jpg")

In [15]:
from tqdm import tqdm

In [ ]:
for file in tqdm(
    glob.glob("/home/setupishe/datasets/VOC/images/**/*.jpg", recursive=True)
):
    to_folder = "val" if "test2007" in file else "train"
    from_folder_name = file.split("/")[-2]

    for func in [lambda x: x, jpg2txt]:
        to_path = func(file).replace(from_folder_name, to_folder)
        shutil.copy(func(file), to_path)

100%|██████████| 21503/21503 [00:01<00:00, 13256.40it/s]


In [ ]:
for file in glob.glob("/home/setupishe/datasets/VOC/*txt"):
    with open(file, "r") as f:
        lines = f.readlines()
    new_lines = []
    for line in lines:
        from_folder_name = line.split("/")[-2]
        to_folder = "val" if "test2007" in line else "train"
        new_lines.append(line.replace(from_folder_name, to_folder))
    with open(file, "w") as f:
        f.writelines(new_lines)

In [ ]:
with open("/home/setupishe/datasets/VOC/val.txt", "r") as f:
    files = [x.rstrip("\n") for x in f.readlines()]
    for file in files:
        full_path = os.path.join("/home/setupishe/datasets/VOC", file)
        if not os.path.getsize(full_path):
            print(file)

In [5]:
with open("/home/setupishe/datasets/coco/train2017_0.2.txt", "r") as f:
    lines = f.readlines()

In [ ]:
weights = "/home/setupishe/ultralytics/runs/detect/confidences_0.6/weights/best.pt"

conf_path = "/".join(weights.split("/")[:-2])
conf_path = "/home/setupishe/ultralytics/runs/detect/temp6" + "/best_conf.txt"

In [ ]:
with open(conf_path, "r") as f:
    conf = float(f.readline())

In [7]:
conf

0.46446446446446443

In [6]:
names = [os.path.basename(x)[:-1] for x in lines]

In [9]:
folder_name = "imgs_0.2"

In [10]:
force_mkdir(folder_name)

In [8]:
def get_shape(img_path):
    with Image.open(img_path) as img:
        width, height = img.size
    return height, width

In [9]:
def txt2jpg(inp):
    return inp.replace("/labels", "/images").replace(".txt", ".jpg")

In [10]:
def seg2bbox(filepath, to_path):
    with open(filepath, "r") as f:
        lines = [x.rstrip("\n") for x in f.readlines()]
    new_lines = []
    for line in lines:
        lst = line.split()
        new_line = [lst[0]]

        coords = [float(x) for x in lst[1:]]
        h, w = get_shape(txt2jpg(filepath))
        first = [x for i, x in enumerate(coords) if i % 2 == 0]
        second = [x for i, x in enumerate(coords) if i % 2 == 1]
        for i in range(len(first)):
            first[i] = int(first[i] * w)
            second[i] = int(second[i] * h)

        xmin = min(first)
        xmax = max(first)
        ymin = min(second)
        ymax = max(second)

        width = (xmax - xmin) / w
        height = (ymax - ymin) / h
        xc = (xmax + xmin) / w / 2
        yc = (ymax + ymin) / h / 2

        new_lines.append(
            " ".join([str(item) for item in [lst[0], xc, yc, width, height]]) + "\n"
        )
    with open(to_path, "w") as f:
        f.writelines(new_lines)

In [12]:
filelist = glob.glob("/home/setupishe/datasets/coco/images/train2017/*jpg")
for file in tqdm(filelist):
    label_file = file.replace("images", "labels").replace("jpg", "txt")
    if os.path.exists(label_file):
        seg2bbox(label_file, file.replace("jpg", "txt"))
    else:
        os.mknod(file.replace("txt"))

  0%|          | 0/118287 [00:00<?, ?it/s]

100%|██████████| 118287/118287 [01:10<00:00, 1677.78it/s]


### remainder embeds

In [33]:
folder_name = "remainder_imgs_0.2"

In [34]:
force_mkdir(folder_name)

In [35]:
filelist = glob.glob("/home/setupishe/datasets/coco/images/train2017/*jpg")
for file in tqdm(filelist):
    if os.path.basename(file) not in names:
        shutil.copy(file, os.path.join(folder_name, os.path.basename(file)))
        label_file = file.replace("images", "labels").replace("jpg", "txt")
        if os.path.exists(label_file):
            seg2bbox(
                label_file, os.path.join(folder_name, os.path.basename(label_file))
            )
        else:
            os.mknod(label_file)

100%|██████████| 118287/118287 [01:48<00:00, 1090.82it/s]


запускаем `produce_detection_embeddings.py`

запускаем `preprocess_embedding_pool.py`

In [2]:
for file in glob.glob("remainder_embeds_reduced_0.2/**/*npy", recursive=True):
    if len(os.path.basename(file).split("_")) == 2:
        os.remove(file)

In [ ]:
for file in glob.glob("rembeds_reduced_0.2/**/*npy", recursive=True):
    if len(os.path.basename(file).split("_")) == 2:
        os.remove(file)

запускаем `testing_embeds.ipynb`

In [26]:
uniq = set()
for file in os.listdir("test_folder"):
    uniq.add(file.split("_")[0])

In [27]:
len(uniq)

13332

In [36]:
target_num = 118287 * 0.1

In [37]:
not_bgs = [x + ".jpg" for x in random.sample(list(uniq), int(target_num * (1 - 0.008)))]

ищем свободные не использованные фоны

In [38]:
from tqdm import tqdm

In [39]:
free_bgs = []

In [40]:
for file in tqdm(
    glob.glob("/home/setupishe/datasets/coco/labels/train*/**/*txt", recursive=True)
):
    name = os.path.basename(file)
    if not os.path.getsize(file) and name not in names:
        free_bgs.append(file)

100%|██████████| 118287/118287 [00:00<00:00, 399844.65it/s]


In [41]:
bgs = [
    os.path.basename(x).replace("txt", "jpg")
    for x in random.sample(free_bgs, int(target_num * 0.008))
]

In [42]:
with open("/home/setupishe/datasets/coco/train2017_0.3_active.txt", "w") as f:
    f.writelines([f"./images/train2017/{x}\n" for x in names + not_bgs + bgs])

In [1]:
int_frac = 0.4

In [2]:
with open(
    "/home/setupishe/ultralytics/ultralytics/cfg/datasets/coco.yaml", "r"
) as from_file:
    lines = from_file.readlines()

for i, line in enumerate(lines):
    if "train: train2017.txt" in line:
        lines[i] = lines[i].replace("train2017.txt", f"train2017_{int_frac}_active.txt")
with open(
    f"/home/setupishe/ultralytics/ultralytics/cfg/datasets/coco_{int_frac}_active.yaml",
    "w",
) as to_file:
    to_file.writelines(lines)

In [ ]:
import numpy as np
from scipy.ndimage import binary_dilation
import time


# Define the function for dilation
def dilate_3d_spherical(input_array, radius):
    """
    Dilate a 3D binary array using a spherical kernel of a given radius.

    Parameters:
        input_array (numpy.ndarray): 3D binary array to be dilated.
        radius (int): Radius of the spherical kernel.

    Returns:
        numpy.ndarray: Dilated 3D array.
    """
    # Generate the spherical kernel using efficient broadcasting
    size = 2 * radius + 1
    x, y, z = np.ogrid[:size, :size, :size]
    center = radius
    distance = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)
    structuring_element = distance <= radius

    # Perform the dilation
    dilated_array = binary_dilation(input_array, structure=structuring_element)

    return dilated_array


# Create a large 3D binary array for testing
array_size = (512, 512, 512)  # Size of the array
large_array = np.zeros(array_size, dtype=bool)

# Add some random points to the array to act as seeds for dilation
np.random.seed(42)
num_points = 1000  # Number of random points
random_indices = np.random.randint(0, array_size[0], size=(num_points, 3))
for idx in random_indices:
    large_array[tuple(idx)] = True

# Test the dilation function with a spherical radius
radius = 5

start_time = time.time()
dilated_array = dilate_3d_spherical(large_array, radius)
end_time = time.time()

time_taken = end_time - start_time
time_taken

29.452413320541382